![](logo1.jpg)

# **shAI Training 2024 | Level 1**

## Task #6 (End-to-End ML Project {part_2})




## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [110]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)

fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model

In [111]:
# CODE HERE
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(housing_prepared,housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [112]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [113]:
# CODE HERE
some_data_prepared=full_pipeline.transform(some_data)
print("predicted_Values",linreg.predict(some_data_prepared))
print("Actual_Values",list(some_labels))

predicted_Values [181746.54359616 290558.74973505 244957.50017771 146498.51061398
 163230.42393939]
Actual_Values [103000.0, 382100.0, 172600.0, 93400.0, 96500.0]


# measure this regression model’s RMSE on the whole training set
* sing Scikit-Learn’s mean_squared_error() function:

In [114]:
from sklearn.metrics import mean_squared_error
houseprediction_linear=linreg.predict(housing_prepared)#columstransformer_to_categorical_and_numerical_data
lin_RMSE=mean_squared_error(housing_labels,houseprediction_linear)
lin_RMSE=np.sqrt(lin_RMSE)
print("Linear_RMSE",lin_RMSE)


Linear_RMSE 67593.20745775253


In [115]:
# CODE HERE

# judge on the RMSE result for this model
write down your answar

your answer goes here

Medain_Housing_Values: ranges from
$120,000 and $265,000

the predicted value is 67593.20745775253 so we need to re fine tune our model

# Let’s train a Decision Tree Regressor model
## more powerful model

In [116]:
from sklearn.tree import DecisionTreeRegressor

In [117]:
treereg=DecisionTreeRegressor()
treereg=treereg.fit(housing_prepared,housing_labels)


# Now evaluate the model on the training set
* using Scikit-Learn’s mean_squared_error() function:

In [118]:
# CODE HERE
housepredict_tree=treereg.predict(housing_prepared)
tree_RMSE=mean_squared_error(housing_labels,housepredict_tree)
tree_RMSE=np.sqrt(tree_RMSE)
print("Tree_RMSE",tree_RMSE)

Tree_RMSE 0.0


# Explaine this result
write down your answar

The model overfits/memorizes the data instead of learning from it


# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [119]:
from sklearn.model_selection import cross_val_score

In [120]:
# CODE HERE
score=cross_val_score(treereg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
RMSE_tree_score=np.sqrt(-score)


2- display the resultant scores and calculate its Mean and Standard deviation

In [121]:
# CODE HERE
def display_statistocs_score(score):
  print("score",score)
  print("mean",score.mean())
  print("standard deviation",score.std())
display_statistocs_score(RMSE_tree_score)

score [65504.32703608 71409.96592452 67684.36490606 69261.95730893
 72611.53488241 67822.96177251 65628.86995497 68886.17890678
 65157.49665604 69479.37876501]
mean 68344.70361133083
standard deviation 2373.473428466595


3-repaet the same steps to compute the same scores for the Linear Regression  model

*notice the difference between the results of the two models*

In [122]:
# CODE HERE
Lin_score=cross_val_score(linreg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
lin_RMSE_score=np.sqrt(-Lin_score)
display_statistocs_score(lin_RMSE_score)

score [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
mean 67828.38677377408
standard deviation 2468.0913950652275


## Let’s train one last model the RandomForestRegressor.

In [123]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor
forestreg=RandomForestRegressor()
forestreg.fit(housing_prepared,housing_labels)

housepredict_forest=forestreg.predict(housing_prepared)
forestreg_RMSE=mean_squared_error(housing_labels,housepredict_forest)
forestreg_RMSE=np.sqrt(forestreg_RMSE)
print("Forest_RMSE",forestreg_RMSE)

Forest_RMSE 18497.705350015898


# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [124]:
# CODE HERE
score_forest=cross_val_score(forestreg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
forestreg_RMSE=np.sqrt(-score_forest)
display_statistocs_score(forestreg_RMSE)

score [47248.67745922 51646.38453893 49609.68152381 51952.08957055
 52609.51672317 47047.31251868 47640.73986968 50889.25044683
 49403.13296517 50136.7315705 ]
mean 49818.35171865397
standard deviation 1901.9242988817277


# Save every model you experiment with
*using the joblib library*

In [125]:
import joblib
joblib.dump(forestreg_RMSE, "myfirstmodel.pkl")
loading_my_model = joblib.load("myfirstmodel.pkl")

In [126]:
joblib.dump(lin_RMSE_score, "myfirstmodel.pkl")
loading_my_model = joblib.load("myfirstmodel.pkl")

In [127]:
joblib.dump(RMSE_tree_score, "myfirstmodel.pkl")
loading_my_model = joblib.load("myfirstmodel.pkl")

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor
*It may take a long time*

In [128]:
from sklearn.model_selection import GridSearchCV

GridSearchCV: A Scikit-Learn tool for performing grid search with cross-validation. It allows you to specify a set of hyperparameters and their possible values, then evaluates all possible combinations.


In [129]:
# CODE HERE
GridSearch_Param=[{"n_estimators":[3,10,35],"max_features":[2,4,6,8]},
            {"bootstrap":[False],"n_estimators":[3,20],"max_features":[2,3,4]}
]

with the evaluation scores

In [130]:
# CODE HERE
GridSearch=GridSearchCV(forestreg,GridSearch_Param,cv=5,scoring="neg_mean_squared_error",return_train_score=True)
GridSearch.fit(housing_prepared,housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 35]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 20]}],
             return_train_score=True, scoring='neg_mean_squared_error')

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [131]:
# CODE HERE
feature_importances=GridSearch.best_estimator_.feature_importances_
feature_importances

array([7.60125729e-02, 6.84007400e-02, 4.18387024e-02, 1.88923975e-02,
       1.67068608e-02, 1.82025884e-02, 1.52804593e-02, 3.53698035e-01,
       5.82656352e-02, 1.04651634e-01, 6.56364095e-02, 1.58864515e-02,
       1.33713845e-01, 2.66892430e-04, 6.06485486e-03, 6.48192171e-03])

2-display these importance scores next to their corresponding attribute names:

In [132]:
# CODE HERE
Exra_atrribute=["rooms_per_household","population_per_household","bedrooms_per_room"]
Cat_Encoder=full_pipeline.named_transformers_["cat"]
Cat_One_Hot_Atrrib=list(Cat_Encoder.categories_[0])
Attributes=Exra_atrribute+Cat_One_Hot_Atrrib+num_attribs
sorted(zip(feature_importances,Attributes),reverse=True)

[(0.3536980345802008, 'NEAR OCEAN'),
 (0.13371384491182586, 'total_bedrooms'),
 (0.10465163395941661, 'latitude'),
 (0.07601257294197897, 'rooms_per_household'),
 (0.06840073997923078, 'population_per_household'),
 (0.06563640950280777, 'housing_median_age'),
 (0.058265635232382484, 'longitude'),
 (0.041838702365936745, 'bedrooms_per_room'),
 (0.01889239752320255, '<1H OCEAN'),
 (0.018202588359935223, 'ISLAND'),
 (0.016706860803669454, 'INLAND'),
 (0.015886451494640005, 'total_rooms'),
 (0.015280459347735614, 'NEAR BAY'),
 (0.006481921705650006, 'median_income'),
 (0.006064854861661854, 'households'),
 (0.00026689242972532923, 'population')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [140]:
# CODE HERE
from sklearn.model_selection import train_test_split
housing_train, housing_test = train_test_split(housing, test_size=0.2, random_state=42)
Final_Model=GridSearch.best_estimator_

2-run your full_pipeline to transform the data

In [135]:
# CODE HERE
housing_prepared=full_pipeline.transform(housing)
fin_prediction=Final_Model.predict(housing_prepared)
fin_RMSE=mean_squared_error(housing_labels,fin_prediction)

3-evaluate the final model on the test set

In [136]:
# CODE HERE
fin_RMSE=np.sqrt(fin_RMSE)
print("fin_RMSE",fin_RMSE)

fin_RMSE 18943.576422618233


# compute a 95% confidence interval for the generalization error
*using scipy.stats.t.interval():*

In [137]:
from scipy import stats
confidence=0.95
squared_error=(fin_prediction-housing_labels)** 2
FINAL=np.sqrt(stats.t.interval(confidence,len(squared_error)-1,loc=squared_error.mean(),scale=stats.sem(squared_error)))
print(FINAL)

[18496.27075919 19380.56096613]


# Great Job!
# #shAI_Club